## 1. Setup & Load Best Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import warnings
warnings.filterwarnings('ignore')

from fbprophet import Prophet
from statsmodels.tsa.arima.model import ARIMA
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ All libraries imported")

In [ ]:
# Load data and model results
df = pd.read_csv('../data/processed/engineered_features.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

# Load test predictions
test_predictions = pd.read_csv('../outputs/test_predictions.csv')
test_predictions['date'] = pd.to_datetime(test_predictions['date'])

# Load model comparison results
with open('../outputs/model_comparison_results.json', 'r') as f:
    model_results = json.load(f)

best_model_name = max(model_results, key=lambda x: x['R²'])['Model']
print(f"✓ Data loaded. Best model: {best_model_name}")
print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")

## 2. Generate Future Forecasts (12-Month)

In [ ]:
# Define forecast horizon (12 months ahead)
forecast_days = 365
last_date = df['date'].max()
forecast_dates = pd.date_range(start=last_date + timedelta(days=1), periods=forecast_days, freq='D')

print(f"Forecast Period: {forecast_dates[0].date()} to {forecast_dates[-1].date()}")
print(f"Total forecast days: {len(forecast_dates)}")

In [ ]:
# Train Prophet on full dataset for forecasting
try:
    prophet_train = df[['date', 'sales']].copy()
    prophet_train.columns = ['ds', 'y']
    
    # Add additional regressors if available
    if 'is_weekend' in df.columns:
        prophet_train['is_weekend'] = df['is_weekend'].values
    
    print("Training Prophet on full dataset for forecasting...")
    prophet_model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10,
        interval_width=0.95
    )
    
    if 'is_weekend' in df.columns:
        prophet_model.add_regressor('is_weekend')
    
    prophet_model.fit(prophet_train)
    
    # Create future dataframe
    future = prophet_model.make_future_dataframe(periods=forecast_days)
    
    # Add regressor values for future
    if 'is_weekend' in df.columns:
        # Calculate is_weekend for future dates
        future['is_weekend'] = (future['ds'].dt.dayofweek >= 5).astype(int)
    
    # Generate forecast
    forecast = prophet_model.predict(future)
    
    # Extract forecast for future period only
    future_forecast = forecast[forecast['ds'] > last_date][['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
    future_forecast.columns = ['date', 'forecast', 'forecast_lower', 'forecast_upper']
    future_forecast['date'] = future_forecast['date'].dt.date
    
    # Ensure forecasts are positive
    future_forecast['forecast'] = future_forecast['forecast'].clip(lower=0)
    future_forecast['forecast_lower'] = future_forecast['forecast_lower'].clip(lower=0)
    future_forecast['forecast_upper'] = future_forecast['forecast_upper'].clip(lower=0)
    
    print(f"✓ Generated {len(future_forecast)} days of forecasts")
    print(f"\nForecast sample:")
    print(future_forecast.head(10))
    
except Exception as e:
    print(f"⚠ Forecast error: {str(e)}")
    future_forecast = None

## 3. Monthly Forecast Summary

In [ ]:
if future_forecast is not None:
    # Convert date back to datetime for grouping
    future_forecast['date'] = pd.to_datetime(future_forecast['date'])
    
    # Monthly aggregation
    monthly_forecast = future_forecast.copy()
    monthly_forecast['year_month'] = monthly_forecast['date'].dt.to_period('M')
    
    monthly_summary = monthly_forecast.groupby('year_month').agg({
        'forecast': 'sum',
        'forecast_lower': 'sum',
        'forecast_upper': 'sum'
    }).reset_index()
    
    print("\n=== MONTHLY FORECAST SUMMARY ===")
    print(monthly_summary.to_string(index=False))
    
    # Calculate statistics
    total_forecast = future_forecast['forecast'].sum()
    avg_daily_forecast = future_forecast['forecast'].mean()
    
    print(f"\nForecast Statistics:")
    print(f"  Total 12-month forecast: ${total_forecast:,.2f}")
    print(f"  Average daily forecast: ${avg_daily_forecast:,.2f}")
    print(f"  Max daily forecast: ${future_forecast['forecast'].max():,.2f}")
    print(f"  Min daily forecast: ${future_forecast['forecast'].min():,.2f}")

## 4. Visualization: Historical + Forecast

In [ ]:
if future_forecast is not None:
    fig, ax = plt.subplots(figsize=(16, 7))
    
    # Historical data
    ax.plot(df['date'], df['sales'], label='Historical Sales', color='steelblue', linewidth=2)
    
    # Forecast
    future_forecast_sorted = future_forecast.sort_values('date')
    ax.plot(future_forecast_sorted['date'], future_forecast_sorted['forecast'], 
            label='12-Month Forecast', color='coral', linewidth=2, linestyle='--')
    
    # Confidence interval
    ax.fill_between(future_forecast_sorted['date'], 
                     future_forecast_sorted['forecast_lower'],
                     future_forecast_sorted['forecast_upper'],
                     color='coral', alpha=0.2, label='95% Confidence Interval')
    
    # Split line
    ax.axvline(last_date, color='red', linestyle='--', alpha=0.5, linewidth=2, label='Forecast Start')
    
    ax.set_title('Historical Sales & 12-Month Forecast', fontsize=14, fontweight='bold')
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Sales', fontsize=12)
    ax.legend(fontsize=11, loc='upper left')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../outputs/08_forecast_visualization.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✓ Saved: 08_forecast_visualization.png")

## 5. Prepare Data for Power BI

In [ ]:
# Combine historical and forecast data
historical_for_bi = df[['date', 'sales']].copy()
historical_for_bi['data_type'] = 'Historical'
historical_for_bi['forecast_lower'] = np.nan
historical_for_bi['forecast_upper'] = np.nan
historical_for_bi.columns = ['date', 'sales', 'data_type', 'forecast_lower', 'forecast_upper']

if future_forecast is not None:
    forecast_for_bi = future_forecast.copy()
    forecast_for_bi['data_type'] = 'Forecast'
    forecast_for_bi['sales'] = forecast_for_bi['forecast']
    forecast_for_bi = forecast_for_bi[['date', 'sales', 'data_type', 'forecast_lower', 'forecast_upper']]
    
    # Combine
    bi_data = pd.concat([historical_for_bi, forecast_for_bi], ignore_index=True)
    bi_data['date'] = pd.to_datetime(bi_data['date'])
    bi_data = bi_data.sort_values('date').reset_index(drop=True)
    
    # Add time dimensions for Power BI
    bi_data['year'] = bi_data['date'].dt.year
    bi_data['month'] = bi_data['date'].dt.month
    bi_data['quarter'] = bi_data['date'].dt.quarter
    bi_data['week'] = bi_data['date'].dt.isocalendar().week
    bi_data['day_of_week'] = bi_data['date'].dt.day_name()
    bi_data['day_of_month'] = bi_data['date'].dt.day
    bi_data['month_name'] = bi_data['date'].dt.month_name()
    
    # Save for Power BI
    bi_data.to_csv('../outputs/powerbi_data.csv', index=False)
    print(f"✓ Saved Power BI data: {len(bi_data)} rows")
    print(f"\nData summary:")
    print(f"  Historical records: {len(historical_for_bi)}")
    print(f"  Forecast records: {len(forecast_for_bi)}")
    print(f"\nFirst few rows:")
    print(bi_data.head(10))

## 6. Business Insights & Recommendations

In [ ]:
print("\n" + "="*60)
print("SALES FORECASTING ANALYSIS - KEY INSIGHTS")
print("="*60)

# 1. Historical Trends
print("\n1. HISTORICAL PERFORMANCE")
print("-" * 60)
hist_total = df['sales'].sum()
hist_avg = df['sales'].mean()
hist_std = df['sales'].std()
hist_days = len(df)

print(f"  • Total Historical Sales: ${hist_total:,.2f}")
print(f"  • Average Daily Sales: ${hist_avg:,.2f}")
print(f"  • Daily Volatility (Std Dev): ${hist_std:,.2f}")
print(f"  • Analysis Period: {hist_days} days")

# 2. Trend Direction
print("\n2. TREND DIRECTION")
print("-" * 60)
first_quarter = df['sales'].iloc[:len(df)//4].mean()
last_quarter = df['sales'].iloc[-len(df)//4:].mean()
trend_direction = ((last_quarter - first_quarter) / first_quarter) * 100

print(f"  • First Quarter Avg: ${first_quarter:,.2f}")
print(f"  • Last Quarter Avg: ${last_quarter:,.2f}")
print(f"  • Trend: {'📈 UPWARD' if trend_direction > 0 else '📉 DOWNWARD'} ({trend_direction:+.1f}%)")

In [ ]:
# 3. Forecast Summary
if future_forecast is not None:
    print("\n3. FORECAST PROJECTIONS")
    print("-" * 60)
    
    forecast_total = future_forecast['forecast'].sum()
    forecast_avg = future_forecast['forecast'].mean()
    forecast_vs_hist = ((forecast_avg - hist_avg) / hist_avg) * 100
    
    print(f"  • 12-Month Forecast Total: ${forecast_total:,.2f}")
    print(f"  • Average Daily Forecast: ${forecast_avg:,.2f}")
    print(f"  • Vs Historical Avg: {forecast_vs_hist:+.1f}%")
    
    if forecast_vs_hist > 0:
        print(f"  • Expected Growth: ${(forecast_avg - hist_avg) * 365:,.2f} annually")
    else:
        print(f"  • Expected Decline: ${(hist_avg - forecast_avg) * 365:,.2f} annually")

# 4. Seasonality
print("\n4. SEASONAL PATTERNS")
print("-" * 60)
monthly_avg = df.groupby('month')['sales'].mean()
peak_month = monthly_avg.idxmax()
low_month = monthly_avg.idxmin()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

peak_value = monthly_avg[peak_month]
low_value = monthly_avg[low_month]
seasonality_strength = ((peak_value - low_value) / low_value) * 100

print(f"  • Peak Season: {month_names[peak_month-1]} (${peak_value:,.2f} avg)")
print(f"  • Low Season: {month_names[low_month-1]} (${low_value:,.2f} avg)")
print(f"  • Seasonality Strength: {seasonality_strength:.1f}%")

# Day of week
dow_avg = df.groupby('day_of_week')['sales'].mean()
peak_dow = dow_avg.idxmax()
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
print(f"  • Strongest Day: {day_names[peak_dow]} (${dow_avg[peak_dow]:,.2f} avg)")

In [ ]:
# 5. Model Performance
print("\n5. MODEL PERFORMANCE")
print("-" * 60)
best_result = max(model_results, key=lambda x: x['R²'])

print(f"  • Best Model: {best_result['Model']}")
print(f"  • Mean Absolute Error (MAE): ${best_result['MAE']:,.2f}")
print(f"  • Root Mean Squared Error (RMSE): ${best_result['RMSE']:,.2f}")
print(f"  • Mean Absolute % Error (MAPE): {best_result['MAPE']:.2%}")
print(f"  • R² Score: {best_result['R²']:.4f}")

# 6. Business Recommendations
print("\n6. STRATEGIC RECOMMENDATIONS")
print("-" * 60)

recommendations = []

# Trend-based
if trend_direction > 5:
    recommendations.append("✅ INCREASE INVENTORY: Strong upward trend detected. Prepare for higher demand.")
elif trend_direction < -5:
    recommendations.append("⚠️  REDUCE INVENTORY: Downward trend detected. Optimize stock levels.")
else:
    recommendations.append("📊 MAINTAIN CURRENT LEVELS: Sales trend is stable.")

# Seasonality-based
if seasonality_strength > 20:
    recommendations.append(f"📈 SEASONAL PLANNING: Strong seasonality detected. Plan peak inventory for {month_names[peak_month-1]}.")
    recommendations.append(f"💰 PROMOTION STRATEGY: Consider discounts/promotions during low season ({month_names[low_month-1]}).")

# Day-of-week based
if peak_dow in [4, 5, 6]:  # Weekend
    recommendations.append(f"🛍️  STAFFING: Increase staff on {day_names[peak_dow]}s (highest sales day).")

# Forecast-based
if future_forecast is not None:
    if forecast_vs_hist > 10:
        recommendations.append(f"📦 CAPACITY PLANNING: Forecasted growth of {forecast_vs_hist:.1f}%. Plan for increased operations.")
    elif forecast_vs_hist < -10:
        recommendations.append(f"💼 COST MANAGEMENT: Forecasted decline of {abs(forecast_vs_hist):.1f}%. Optimize operations for lower volumes.")

# Model confidence
if best_result['R²'] > 0.85:
    recommendations.append(f"✨ HIGH CONFIDENCE: Model R² of {best_result['R²']:.2%} provides reliable forecasts.")
elif best_result['R²'] > 0.7:
    recommendations.append(f"✓ MODERATE CONFIDENCE: Model R² of {best_result['R²']:.2%}. Monitor actual vs forecast.")
else:
    recommendations.append(f"⚠️  LOWER CONFIDENCE: Model R² of {best_result['R²']:.2%}. Use cautiously with domain expertise.")

for rec in recommendations:
    print(f"  • {rec}")

## 7. Export Summary Report

In [ ]:
# Create summary report
report = {
    'Analysis_Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'Historical_Period': {
        'Start_Date': df['date'].min().strftime('%Y-%m-%d'),
        'End_Date': df['date'].max().strftime('%Y-%m-%d'),
        'Total_Days': int(hist_days),
        'Total_Sales': float(hist_total),
        'Average_Daily_Sales': float(hist_avg),
        'Daily_Volatility_StdDev': float(hist_std)
    },
    'Trend_Analysis': {
        'Direction': 'Upward' if trend_direction > 0 else 'Downward',
        'Percentage_Change': float(trend_direction),
        'First_Quarter_Avg': float(first_quarter),
        'Last_Quarter_Avg': float(last_quarter)
    },
    'Seasonality': {
        'Peak_Month': month_names[peak_month-1],
        'Peak_Monthly_Avg': float(peak_value),
        'Low_Month': month_names[low_month-1],
        'Low_Monthly_Avg': float(low_value),
        'Seasonality_Strength_Percent': float(seasonality_strength),
        'Peak_Day_Of_Week': day_names[peak_dow],
        'Peak_Day_Avg': float(dow_avg[peak_dow])
    },
    'Model_Performance': {
        'Best_Model': best_result['Model'],
        'MAE': float(best_result['MAE']),
        'RMSE': float(best_result['RMSE']),
        'MAPE': float(best_result['MAPE']),
        'R_Squared': float(best_result['R²'])
    },
    'Forecast': {
        'Forecast_Period': '12 Months',
        'Total_Forecast': float(forecast_total) if future_forecast is not None else None,
        'Average_Daily_Forecast': float(forecast_avg) if future_forecast is not None else None,
        'Forecast_vs_Historical_Percent': float(forecast_vs_hist) if future_forecast is not None else None
    },
    'Recommendations': recommendations
}

# Save report
with open('../outputs/analysis_summary_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print("\n" + "="*60)
print("✓ Analysis complete!")
print("="*60)
print(f"\nOutput Files Generated:")
print(f"  1. powerbi_data.csv - Historical + Forecast data")
print(f"  2. analysis_summary_report.json - Executive summary")
print(f"  3. test_predictions.csv - Model predictions")
print(f"  4. model_comparison_results.json - Model metrics")
print(f"\nVisualization Files:")
print(f"  ✓ 08_forecast_visualization.png")
print(f"\nNext Steps:")
print(f"  1. Open 'powerbi_data.csv' in Power BI Desktop")
print(f"  2. Create visualizations using the forecast data")
print(f"  3. Share insights with stakeholders")
print("\n" + "="*60)

## 8. Final Summary Statistics

In [ ]:
# Create comprehensive summary table
summary_metrics = {
    'Metric': [
        'Data Points (Historical)',
        'Date Range',
        'Total Sales',
        'Average Daily Sales',
        'Sales Volatility (σ)',
        'Peak Month',
        'Low Month',
        'Best Forecasting Model',
        'Model Accuracy (R²)',
        'Forecast Confidence (MAPE)',
        '12-Month Forecast Total',
        'Expected Growth Rate',
        'Peak Seasonality',
    ],
    'Value': [
        f"{int(hist_days)} days",
        f"{df['date'].min().date()} to {df['date'].max().date()}",
        f"${hist_total:,.0f}",
        f"${hist_avg:,.0f}",
        f"${hist_std:,.0f}",
        f"{month_names[peak_month-1]} (${peak_value:,.0f})",
        f"{month_names[low_month-1]} (${low_value:,.0f})",
        best_result['Model'],
        f"{best_result['R²']:.2%}",
        f"{best_result['MAPE']:.2%}",
        f"${forecast_total:,.0f}" if future_forecast is not None else "N/A",
        f"{forecast_vs_hist:+.1f}%" if future_forecast is not None else "N/A",
        f"{seasonality_strength:.1f}%",
    ]
}

summary_df = pd.DataFrame(summary_metrics)
print("\n" + "="*80)
print("EXECUTIVE SUMMARY - KEY METRICS")
print("="*80)
print(summary_df.to_string(index=False))
print("="*80)